# batch makes all queries to get top 40 products relevant for each query

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

In [5]:
import sys
sys.executable

'/opt/conda/envs/p312/bin/python'

In [6]:
model = 'gemini'
locale = 'us'

In [8]:
q = pd.read_parquet(f'{model}/queries-{locale}.parquet')

In [9]:
p = pd.read_parquet(f'{model}/products-{locale}.parquet')

In [10]:
q.shape, p.shape

((97345, 2), (1215851, 2))

In [11]:
qe = np.stack(q.embeddings.values)
pe = np.stack(p.embeddings.values)
qe.shape, pe.shape

((97345, 768), (1215851, 768))

In [12]:
def get_dotp_closests_idxs(q_embedding,p_embeddings_matrix, topk=40):
    return np.argsort( p_embeddings_matrix.dot(q_embedding) )[-topk:][::-1]


In [13]:
1

1

In [ ]:
qnn = Parallel(n_jobs=-1, verbose=5, prefer='threads')(delayed(get_dotp_closests_idxs)(qi,pe) for qi in qe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done 520 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 754 tasks      | elapsed:   48.6s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1330 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 2050 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 2464 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 2914 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 3400 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 3922 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 4480 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 5074 tasks   

In [ ]:
q['top40_products'] = [list(p.index[qnni]) for qnni in tqdm(qnn)]

In [ ]:
qr = []
for query_id, qi in tqdm(q.iterrows()):
    for product_id in qi.top40_products.values:
        qr.append([query_id, product_id])
    

In [ ]:
pd.DataFrame(qr, columns = ['query_id', 'product_id']).to_csv(f'{model}/embeddings_dotp_ranking_{locale}.csv', index=False)